<h1>Flu Shot Learning: Predict H1N1 and Seasonal Flu Vaccines</h1>

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## Load data

In [2]:
train = pd.read_csv('./data/training_set_features.csv', index_col='respondent_id')

In [3]:
test = pd.read_csv('./data/test_set_features.csv', index_col='respondent_id')

In [4]:
labels = pd.read_csv('./data/training_set_labels.csv', index_col='respondent_id')

### Imputation strategy

In [5]:
num_cols = train.select_dtypes('number').columns

In [6]:
cat_cols = ['race', 'sex', 
       'marital_status', 'rent_or_own',  'hhs_geo_region',
       'census_msa', 'employment_industry', 'employment_occupation']

In [7]:
ord_cols = ['age_group', 'education',  'income_poverty',
        'employment_status']

In [8]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from category_encoders import OrdinalEncoder as oe
from catboost import CatBoostClassifier
from catboost import Pool, cv
from sklearn.metrics import roc_curve, roc_auc_score
import optuna

c:\Users\40107588\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Impute train

In [9]:
# Categorical columns None
for col in (cat_cols+ord_cols):
    train[col] = train[col].fillna(value='None')

In [10]:
for col in num_cols:
    train[col] = train[col].fillna(value=-1)

#### Impute test

In [11]:
# Categorical columns None
for col in (cat_cols+ord_cols):
    test[col] = test[col].fillna(value='None')

In [12]:
for col in num_cols:
    test[col] = test[col].fillna(value=-1)

### Train test split

In [13]:
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_test, y_train, y_test = train_test_split( train, labels, test_size=0.3, random_state=68)

In [15]:
# Get the index number of categorical features
categorical_features_indices = np.where(X_train.dtypes != float)[0]

### Optimize with Optuna and cross validation

Using just the selected features idetified in the previous notebook, led to poor perfomances; it seems to be better to use all the columns. CatBoost can easily deal with useless features not using them. 

In [16]:
train_dataset = Pool(data=X_train,
                     label=y_train.h1n1_vaccine,
                     cat_features = categorical_features_indices)

In [17]:
def objective(trial):
    param = {
        'iterations':trial.suggest_categorical('iterations', [100,200,300,500,1000,1200,1500]), #number of boosting iterations (trees) to train
        'learning_rate':trial.suggest_float("learning_rate", 0.001, 0.3), #step size at each iteration while moving towards a minimum of the loss function
        'random_strength':trial.suggest_int("random_strength", 1,10), # strength of the randomness in tree building
        'bagging_temperature':trial.suggest_int("bagging_temperature", 0,10), #degree of randomness in the selection of training data
        'max_bin':trial.suggest_categorical('max_bin', [4,5,6,8,10,20,30]), #maximum number of bins used for feature discretization
        'grow_policy':trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']), #policy used to grow the trees
        'min_data_in_leaf':trial.suggest_int("min_data_in_leaf", 1,10), #Minimum number of samples in a leaf node
        'od_type' : "Iter", #Type of overfitting detector
        'od_wait' : 100, #Number of iterations to wait before stopping the training process if no improvement
        "depth": trial.suggest_int("max_depth", 2,10), #Maximum depth of the trees
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100), #L2 regularization coefficient for leaf weights
        'one_hot_max_size':trial.suggest_categorical('one_hot_max_size', [5,10,12,100,500,1024]), #Maximum number of categories for one-hot encoding 
        'custom_metric' : ['AUC'],
        "loss_function": "Logloss",
        'auto_class_weights':trial.suggest_categorical('auto_class_weights', ['Balanced', 'SqrtBalanced']), #Strategy for automatically adjusting class weights
        }

    scores = cv(train_dataset,
            param,
            fold_count=5, 
            early_stopping_rounds=10,         
            plot=False, verbose=False)

    return scores['test-AUC-mean'].max()

Even though CatBoost is quite fast and I used early stopping, it's going to run 100 fits so it takes a lot of time. You can avoid re-running the optimisation and copy the parameters from the print report replacing `**trial.params` when calling CatBoost

In [18]:
sampler = optuna.samplers.TPESampler(seed=68)  # Make the sampler behave in a deterministic way.
study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=100)

[I 2024-07-26 11:15:57,185] A new study created in memory with name: no-name-5ab0c57d-3beb-4395-bb2c-b844e41e6711
C:\Users\40107588\AppData\Local\Temp\ipykernel_3444\1656202261.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100), #L2 regularization coefficient for leaf weights


Training on fold [0/5]

bestTest = 0.4269336823
bestIteration = 569

Training on fold [1/5]

bestTest = 0.4240454756
bestIteration = 497

Training on fold [2/5]

bestTest = 0.4324963035
bestIteration = 536

Training on fold [3/5]

bestTest = 0.4286695316
bestIteration = 546

Training on fold [4/5]


[I 2024-07-26 11:16:59,200] Trial 0 finished with value: 0.863647892026189 and parameters: {'iterations': 1500, 'learning_rate': 0.029356482739949695, 'random_strength': 8, 'bagging_temperature': 10, 'max_bin': 6, 'grow_policy': 'SymmetricTree', 'min_data_in_leaf': 1, 'max_depth': 4, 'l2_leaf_reg': 0.001991194871120998, 'one_hot_max_size': 100, 'auto_class_weights': 'SqrtBalanced'}. Best is trial 0 with value: 0.863647892026189.



bestTest = 0.4289279999
bestIteration = 556

Training on fold [0/5]


C:\Users\40107588\AppData\Local\Temp\ipykernel_3444\1656202261.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100), #L2 regularization coefficient for leaf weights


...


bestTest = 0.4660103278
bestIteration = 54

Training on fold [1/5]

bestTest = 0.4567015524
bestIteration = 94

Training on fold [2/5]

bestTest = 0.4706253926
bestIteration = 92

Training on fold [3/5]

bestTest = 0.4695637146
bestIteration = 66

Training on fold [4/5]


[I 2024-07-26 12:45:48,748] Trial 47 finished with value: 0.8606778019285182 and parameters: {'iterations': 1200, 'learning_rate': 0.08564940608004865, 'random_strength': 1, 'bagging_temperature': 4, 'max_bin': 30, 'grow_policy': 'Lossguide', 'min_data_in_leaf': 5, 'max_depth': 9, 'l2_leaf_reg': 0.00029726225174476226, 'one_hot_max_size': 10, 'auto_class_weights': 'Balanced'}. Best is trial 42 with value: 0.8670574726329783.



bestTest = 0.4730740481
bestIteration = 62

Training on fold [0/5]


C:\Users\40107588\AppData\Local\Temp\ipykernel_3444\1656202261.py:13: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100), #L2 regularization coefficient for leaf weights



bestTest = 0.4599902051
bestIteration = 209

Training on fold [1/5]

bestTest = 0.4520625107
bestIteration = 247

Training on fold [2/5]

bestTest = 0.4588034204
bestIteration = 258

Training on fold [3/5]


In [ ]:
print("Number of finished trials: {}".format(len(study.trials)))
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}={},".format(key, value))


: 

### Check the model

In [ ]:
final_model = CatBoostClassifier(verbose=False,  cat_features=categorical_features_indices, 
                          **trial.params)

: 

In [ ]:
final_model.fit(X_train, y_train.h1n1_vaccine)

: 

In [ ]:
predictions_h1 = final_model.predict_proba(X_test)

: 

In [ ]:
predictions_h1 = predictions_h1[:,1].reshape(-1,1)

: 

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

: 

In [ ]:
def plot_roc(y_true, y_score, label_name, ax):
    fpr, tpr, thresholds = roc_curve(y_true, y_score)
    ax.plot(fpr, tpr)
    ax.plot([0, 1], [0, 1], color='grey', linestyle='--')
    ax.set_ylabel('TPR')
    ax.set_xlabel('FPR')
    ax.set_title(
        f"{label_name}: AUC = {roc_auc_score(y_true, y_score):.4f}"
    )

: 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_roc(
    y_test['h1n1_vaccine'], 
    predictions_h1, 
    'h1n1_vaccine',
    ax=ax
)

: 

In [ ]:
roc_auc_score(y_test.h1n1_vaccine, predictions_h1)

: 

## Second part

In [ ]:
train_dataset_se = Pool(data=X_train,
                     label=y_train.seasonal_vaccine,
                     cat_features = categorical_features_indices)

: 

I made a second objective function in case I'd want to change some tuning

In [ ]:
def objective2(trial):
    param = {
        'iterations':trial.suggest_categorical('iterations', [100,200,300,500,1000,1200,1500]),
        'learning_rate':trial.suggest_float("learning_rate", 0.001, 0.3),
        'random_strength':trial.suggest_int("random_strength", 1,10),
        'bagging_temperature':trial.suggest_int("bagging_temperature", 0,10),
        'max_bin':trial.suggest_categorical('max_bin', [4,5,6,8,10,20,30]),
        'grow_policy':trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise', 'Lossguide']),
        'min_data_in_leaf':trial.suggest_int("min_data_in_leaf", 1,10),
        'od_type' : "Iter",
        'od_wait' : 100,
        "depth": trial.suggest_int("max_depth", 2,10),
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 1e-8, 100),
         'one_hot_max_size':trial.suggest_categorical('one_hot_max_size', [5,10,12,100,500,1024]),
        'custom_metric' : ['AUC'],
        "loss_function": "Logloss",
        'auto_class_weights':trial.suggest_categorical('auto_class_weights', ['Balanced', 'SqrtBalanced']),
        }

    scores = cv(train_dataset_se,
            param,
            fold_count=5, 
            early_stopping_rounds=10,         
            plot=False, verbose=False) 

    return scores['test-AUC-mean'].max()

: 

In [ ]:
sampler = optuna.samplers.TPESampler(seed=68)  # Make the sampler behave in a deterministic way.
study2 = optuna.create_study(direction="maximize", sampler=sampler)
study2.optimize(objective2, n_trials=100)

: 

In [ ]:
print("Number of finished trials: {}".format(len(study2.trials)))
print("Best trial:")
trial2 = study2.best_trial
print("  Value: {}".format(trial2.value))
print("  Params: ")
for key, value in trial2.params.items():
    print("    {}={},".format(key, value))


: 

In [ ]:
Number of finished trials: 100
Best trial:
  Value: 0.8679538793376752
  Params: 
    iterations=500,
    learning_rate=0.05745075659543725,
    random_strength=4,
    bagging_temperature=8,
    max_bin=5,
    grow_policy=Lossguide,
    min_data_in_leaf=7,
    max_depth=6,
    l2_leaf_reg=11.323094517862078,
    one_hot_max_size=10,
    auto_class_weights=Balanced,

: 

In [ ]:
final_model_se = CatBoostClassifier(verbose=False,  cat_features=categorical_features_indices, 
                                    **trial2.params)

: 

In [ ]:
final_model_se.fit(X_train, y_train.seasonal_vaccine)

: 

In [ ]:
predictions_se = final_model_se.predict_proba(X_test)

: 

In [ ]:
predictions_se = predictions_se[:,1].reshape(-1,1)

: 

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 8))
plot_roc(
    y_test['seasonal_vaccine'], 
    predictions_se, 
    'seasonal_vaccine',
    ax=ax
)

: 

In [ ]:
roc_auc_score(y_test.seasonal_vaccine, predictions_se)

: 

## Combined score

In [ ]:
# Let's see the score combined of the two best predictions
roc_auc_score(y_test, np.hstack((predictions_h1, predictions_se)))

: 

## Retrain on full dataset

#### Seasonal

In [ ]:
final_model_se.fit(train, labels.seasonal_vaccine)

: 

In [ ]:
final_se = final_model_se.predict_proba(test)

: 

In [ ]:
final_se = final_se[:,1].reshape(-1,1)

: 

#### H1N1

In [ ]:
final_model.fit(train, labels.h1n1_vaccine)

: 

In [ ]:
final_h1 = final_model.predict_proba(test)

: 

In [ ]:
final_h1 = final_h1[:,1].reshape(-1,1)

: 

## Make submission

In [ ]:
submission_df = pd.read_csv("./submission_format.csv", 
                            index_col="respondent_id")

: 

In [ ]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test.index.values, 
                              submission_df.index.values)

# Save predictions to submission data frame
submission_df["h1n1_vaccine"] = final_h1
submission_df["seasonal_vaccine"] = final_se

submission_df.head()

: 

In [ ]:
date = pd.Timestamp.now().strftime(format='%Y-%m-%d_%H-%M_')
submission_df.to_csv(f'predictions/{date}submssion_catboost_optunacvi.csv', index=True)

: 

The model scored `0.8608` not too bad, much better than the benchmark. There is a lot of room for improvment (my present score is `0.8638`, ranking **7th** on the 16th February 2021). We can verify some more parameters, CatBoost has a lot of [them](https://catboost.ai/docs/concepts/parameter-tuning.html), we may try some feature engineering (I tried some basice ones but the score did not improve so I omitted them in the analysis), change the model (LightGBM, XGBoost for example), try different imputations, increase the rounds for `optuna` a little bit and so on. Spend some time analysing the dataset and the correlation among the variables using EDA notebooks.

© Andrea Dalseno 2021